In [133]:
import numpy as np
import pandas as pd
import sys, os
from random import shuffle
from tqdm import tqdm
import json,pickle
from collections import OrderedDict
from rdkit import Chem
from rdkit.Chem import MolFromSmiles
import networkx as nx
import torch
import torch.nn as nn
from models.gat import GATNet
from models.gat_gcn import GAT_GCN
from models.gcn import GCNNet
from models.ginconv import GINConvNet
from utils import *

In [134]:
f1 = 'Benzos-structures-filteres-by-similarity-to-diazepam-20211216T105015Z-001/Benzos-structures-filteres-by-similarity-to-diazepam/LOWDATAREGIME_molecules_10_0.7_BENZOS_TOP_SIMILAR.txt'
f2 = 'Benzos-structures-filteres-by-similarity-to-diazepam-20211216T105015Z-001/Benzos-structures-filteres-by-similarity-to-diazepam/LOWDATAREGIME_molecules_20_0.7_BENZOS_TOP_SIMILAR.txt'
f3 = 'Benzos-structures-filteres-by-similarity-to-diazepam-20211216T105015Z-001/Benzos-structures-filteres-by-similarity-to-diazepam/LOWDATAREGIME_molecules_30_0.7_BENZOS_TOP_SIMILAR.txt'
f4 = 'Benzos-structures-filteres-by-similarity-to-diazepam-20211216T105015Z-001/Benzos-structures-filteres-by-similarity-to-diazepam/LOWDATAREGIME_molecules_40_0.7_BENZOS_TOP_SIMILAR.txt'
f5 = 'Benzos-structures-filteres-by-similarity-to-diazepam-20211216T105015Z-001/Benzos-structures-filteres-by-similarity-to-diazepam/REINVENT_BENZOS_TOP_SIMILAR.txt'
f6 = 'Benzos-structures-filteres-by-similarity-to-diazepam-20211216T105015Z-001/Benzos-structures-filteres-by-similarity-to-diazepam/TRANSMOL_BENZOS_TOP_SIMILAR.txt'
f7 = 'Benzos-structures-filteres-by-similarity-to-diazepam-20211216T105015Z-001/Benzos-structures-filteres-by-similarity-to-diazepam/TRANSVAE_benzos_pubchem_k_dims_BENZOS_TOP_SIMILAR.txt'
f8 = 'Benzos-structures-filteres-by-similarity-to-diazepam-20211216T105015Z-001/Benzos-structures-filteres-by-similarity-to-diazepam/TRANSVAE_benzos_pubchem_top_dims_BENZOS_TOP_SIMILAR.txt'
f9 = 'Benzos-structures-filteres-by-similarity-to-diazepam-20211216T105015Z-001/Benzos-structures-filteres-by-similarity-to-diazepam/TRANSVAE_benzos_zinc_k_dims_BENZOS_TOP_SIMILAR.txt'
f10 = 'Benzos-structures-filteres-by-similarity-to-diazepam-20211216T105015Z-001/Benzos-structures-filteres-by-similarity-to-diazepam/TRANSVAE_benzos_zinc_top_dims_BENZOS_TOP_SIMILAR.txt'

In [135]:
m1 = 'VitaminDZinc-structures-filteres-by-similarity-to-calcitriol/LOWDATAREGIME_molecules_10_0.7_VDR_TOP_SIMILAR.txt'
m2 = 'VitaminDZinc-structures-filteres-by-similarity-to-calcitriol/LOWDATAREGIME_molecules_20_0.7_VDR_TOP_SIMILAR.txt'
m3 = 'VitaminDZinc-structures-filteres-by-similarity-to-calcitriol/LOWDATAREGIME_molecules_30_0.7_VDR_TOP_SIMILAR.txt'
m4 = 'VitaminDZinc-structures-filteres-by-similarity-to-calcitriol/LOWDATAREGIME_molecules_40_0.7_VDR_TOP_SIMILAR.txt'
m5 = 'VitaminDZinc-structures-filteres-by-similarity-to-calcitriol/REINVENT_VDR_TOP_SIMILAR.txt'
m6 = 'VitaminDZinc-structures-filteres-by-similarity-to-calcitriol/TRANSMOL_VDR_TOP_SIMILAR.txt'
m7 = 'VitaminDZinc-structures-filteres-by-similarity-to-calcitriol/TRANSVAE_vdr_pubchem_k_dims_VDR_TOP_SIMILAR.txt'
m8 = 'VitaminDZinc-structures-filteres-by-similarity-to-calcitriol/TRANSVAE_vdr_pubchem_top_dims_VDR_TOP_SIMILAR.txt'
m9 = 'VitaminDZinc-structures-filteres-by-similarity-to-calcitriol/TRANSVAE_vdr_zinc_k_dims_VDR_TOP_SIMILAR.txt'
m10 = 'VitaminDZinc-structures-filteres-by-similarity-to-calcitriol/TRANSVAE_vdr_zinc_top_dims_VDR_TOP_SIMILAR.txt'

In [136]:
gaba = 'MRKSPGLSDCLWAWILLLSTLTGRSYGQPSLQDELKDNTTVFTRILDRLLDGYDNRLRPGLGERVTEVKTDIFVTSFGPVSDHDMEYTIDVFFRQSWKDERLKFKGPMTVLRLNNLMASKIWTPDTFFHNGKKSVAHNMTMPNKLLRITEDGTLLYTMRLTVRAECPMHLEDFPMDAHACPLKFGSYAYTRAEVVYEWTREPARSVVVAEDGSRLNQYDLLGQTVDSGIVQSSTGEYVVMTTHFHLKRKIGYFVIQTYLPCIMTVILSQVSFWLNRESVPARTVFGVTTVLTMTTLSISARNSLPKVAYATAMDWFIAVCYAFVFSALIEFATVNYFTKRGYAWDGKSVVPEKPKKVKDPLIKKNNTYAPTATSYTPNLARGDPGLATIAKSATIEPKEVKPETKPPEPKKTFNSVSKIDRLSRIAFPLLFGIFNLVYWATYLNREPQLKAPTPHQ'
diazepam = 'CN1C(=O)CN=C(c2ccccc2)c2cc(Cl)ccc21'

In [137]:
vdr = 'MEAMAASTSLPDPGDFDRNVPRICGVCGDRATGFHFNAMTCEGCKGFFRRSMKRKALFTCPFNGDCRITKDNRRHCQACRLKRCVDIGMMKEFILTDEEVQRKREMILKRKEEEALKDSLRPKLSEEQQRIIAILLDAHHKTYDPTYSDFCQFRPPVRVNDGGGSHPSRPNSRHTPSFSGDSSSSCSDHCITSSDMMDSSSFSNLDLSEEDSDDPSVTLELSQLSMLPHLADLVSYSIQKVIGFAKMIPGFRDLTSEDQIVLLKSSAIEVIMLRSNESFTMDDMSWTCGNQDYKYRVSDVTKAGHSLELIEPLIKFQVGLKKLNLHEEEHVLLMAICIVSPDRPGVQDAALIEAIQDRLSNTLQTYIRCRHPPPGSHLLYAKMIQKLADLRSLNEEHSKQYRCLSFQPECSMKLTPLVLEVFGNEIS'
calcitriol = 'CC(CCCC(C)(C)O)C1CCC2C1(CCCC2=CC=C3CC(CC(C3=C)O)O)C'

In [87]:
# df1 = pd.read_csv(f1,header= None)
# df2 = pd.read_csv(f2,header= None)
# df3 = pd.read_csv(f3,header= None)
# df4 = pd.read_csv(f4,header= None)
# df5 = pd.read_csv(f5,header= None)
# df6 = pd.read_csv(f6,header= None)
# df7 = pd.read_csv(f7,header= None)
# df8 = pd.read_csv(f8,header= None)
# df9 = pd.read_csv(f9,header= None)
# df10 = pd.read_csv(f10,header= None)


In [138]:
vdf1 = pd.read_csv('Benzos-structures-filteres-by-similarity-to-diazepam-20211216T105015Z-001/Benzos-structures-filteres-by-similarity-to-diazepam/csv_format/LOWDATAREGIME_molecules_10_0.7_BENZOS_TOP_SIMILAR.csv')
vdf2 =pd.read_csv('Benzos-structures-filteres-by-similarity-to-diazepam-20211216T105015Z-001/Benzos-structures-filteres-by-similarity-to-diazepam/csv_format/LOWDATAREGIME_molecules_20_0.7_BENZOS_TOP_SIMILAR.csv')
vdf3 = pd.read_csv('Benzos-structures-filteres-by-similarity-to-diazepam-20211216T105015Z-001/Benzos-structures-filteres-by-similarity-to-diazepam/csv_format/LOWDATAREGIME_molecules_30_0.7_BENZOS_TOP_SIMILAR.csv')
vdf4 = pd.read_csv('Benzos-structures-filteres-by-similarity-to-diazepam-20211216T105015Z-001/Benzos-structures-filteres-by-similarity-to-diazepam/csv_format/LOWDATAREGIME_molecules_40_0.7_BENZOS_TOP_SIMILAR.csv')
vdf5 = pd.read_csv('Benzos-structures-filteres-by-similarity-to-diazepam-20211216T105015Z-001/Benzos-structures-filteres-by-similarity-to-diazepam/csv_format/REINVENT_BENZOS_TOP_SIMILAR.csv')
vdf6 = pd.read_csv('Benzos-structures-filteres-by-similarity-to-diazepam-20211216T105015Z-001/Benzos-structures-filteres-by-similarity-to-diazepam/csv_format/TRANSMOL_BENZOS_TOP_SIMILAR.csv')
vdf7 = pd.read_csv('Benzos-structures-filteres-by-similarity-to-diazepam-20211216T105015Z-001/Benzos-structures-filteres-by-similarity-to-diazepam/csv_format/TRANSVAE_benzos_pubchem_k_dims_BENZOS_TOP_SIMILAR.csv')
vdf8 = pd.read_csv('Benzos-structures-filteres-by-similarity-to-diazepam-20211216T105015Z-001/Benzos-structures-filteres-by-similarity-to-diazepam/csv_format/TRANSVAE_benzos_pubchem_top_dims_BENZOS_TOP_SIMILAR.csv')
vdf9 = pd.read_csv('Benzos-structures-filteres-by-similarity-to-diazepam-20211216T105015Z-001/Benzos-structures-filteres-by-similarity-to-diazepam/csv_format/TRANSVAE_benzos_zinc_k_dims_BENZOS_TOP_SIMILAR.csv')
vdf10 = pd.read_csv('Benzos-structures-filteres-by-similarity-to-diazepam-20211216T105015Z-001/Benzos-structures-filteres-by-similarity-to-diazepam/csv_format/TRANSVAE_benzos_zinc_top_dims_BENZOS_TOP_SIMILAR.csv')

In [89]:
df1 = pd.read_csv('VitaminDZinc-structures-filteres-by-similarity-to-calcitriol/csv_format/LOWDATAREGIME_molecules_10_0.7_VDR_TOP_SIMILAR.csv')
df2 =pd.read_csv('VitaminDZinc-structures-filteres-by-similarity-to-calcitriol/csv_format/LOWDATAREGIME_molecules_20_0.7_VDR_TOP_SIMILAR.csv')
df3 = pd.read_csv('VitaminDZinc-structures-filteres-by-similarity-to-calcitriol/csv_format/LOWDATAREGIME_molecules_30_0.7_VDR_TOP_SIMILAR.csv')
df4 = pd.read_csv('VitaminDZinc-structures-filteres-by-similarity-to-calcitriol/csv_format/LOWDATAREGIME_molecules_40_0.7_VDR_TOP_SIMILAR.csv')
df5 = pd.read_csv('VitaminDZinc-structures-filteres-by-similarity-to-calcitriol/csv_format/REINVENT_VDR_TOP_SIMILAR.csv')
df6 = pd.read_csv('VitaminDZinc-structures-filteres-by-similarity-to-calcitriol/csv_format/TRANSMOL_VDR_TOP_SIMILAR.csv')
df7 = pd.read_csv('VitaminDZinc-structures-filteres-by-similarity-to-calcitriol/csv_format/TRANSVAE_vdr_pubchem_k_dims_VDR_TOP_SIMILAR.csv')
df8 = pd.read_csv('VitaminDZinc-structures-filteres-by-similarity-to-calcitriol/csv_format/TRANSVAE_vdr_pubchem_top_dims_VDR_TOP_SIMILAR.csv')
df9 = pd.read_csv('VitaminDZinc-structures-filteres-by-similarity-to-calcitriol/csv_format/TRANSVAE_vdr_zinc_k_dims_VDR_TOP_SIMILAR.csv')
df10 = pd.read_csv('VitaminDZinc-structures-filteres-by-similarity-to-calcitriol/csv_format/TRANSVAE_vdr_zinc_top_dims_VDR_TOP_SIMILAR.csv')

In [139]:
vdf1

,Drug,Target,Y
0,CN1C(=O)CN=C(c2ccccc2F)c2cc(Cl)ccc21,MRKSPGLSDCLWAWILLLSTLTGRSYGQPSLQDELKDNTTVFTRIL...,0.0
1,O=C1CN=C(c2ccccc2)c2cc(Cl)ccc2N1,MRKSPGLSDCLWAWILLLSTLTGRSYGQPSLQDELKDNTTVFTRIL...,0.0
2,CN1CCN=C(c2ccccc2)c2cc(Cl)ccc21,MRKSPGLSDCLWAWILLLSTLTGRSYGQPSLQDELKDNTTVFTRIL...,0.0
3,CN1C(=O)CN=C(c2ccccc2)c2cc(N)ccc21,MRKSPGLSDCLWAWILLLSTLTGRSYGQPSLQDELKDNTTVFTRIL...,0.0
4,CN1C(=O)CCN=C(c2ccccc2)c2cc(Cl)ccc21,MRKSPGLSDCLWAWILLLSTLTGRSYGQPSLQDELKDNTTVFTRIL...,0.0
...,...,...,...
483,NC1=NC(C=Cc2ccc3nccnc3c2)CCC1,MRKSPGLSDCLWAWILLLSTLTGRSYGQPSLQDELKDNTTVFTRIL...,0.0
484,O=C(OC1CC1)c1nccc2cc(-c3ccccc3)ccc12,MRKSPGLSDCLWAWILLLSTLTGRSYGQPSLQDELKDNTTVFTRIL...,0.0
485,O=C1NCC1=COc1ccc2cncnc2c1,MRKSPGLSDCLWAWILLLSTLTGRSYGQPSLQDELKDNTTVFTRIL...,0.0
486,COC1CN=c2ccc(Cl)cc2=N1,MRKSPGLSDCLWAWILLLSTLTGRSYGQPSLQDELKDNTTVFTRIL...,0.0


In [91]:
# vdf1 = pd.read_csv(m1,header= None)
# vdf2 = pd.read_csv(m2,header= None)
# vdf3 = pd.read_csv(m3,header= None)
# vdf4 = pd.read_csv(m4,header= None)
# vdf5 = pd.read_csv(m5,header= None)
# vdf6 = pd.read_csv(m6,header= None)
# vdf7 = pd.read_csv(m7,header= None)
# vdf8 = pd.read_csv(m8,header= None)
# vdf9 = pd.read_csv(m9,header= None)
# vdf10 = pd.read_csv(m10,header= None)


In [92]:
# vdf1 = vdf1.rename(columns={0:'Drug'})
# vdf2 = vdf2.rename(columns={0:'Drug'})
# vdf3 = vdf3.rename(columns={0:'Drug'})
# vdf4 = vdf4.rename(columns={0:'Drug'})
# vdf5 = vdf5.rename(columns={0:'Drug'})
# vdf6 = vdf6.rename(columns={0:'Drug'})
# vdf7 = vdf7.rename(columns={0:'Drug'})
# vdf8 = vdf8.rename(columns={0:'Drug'})
# vdf9 = vdf9.rename(columns={0:'Drug'})
# vdf10 = vdf10.rename(columns={0:'Drug'})

In [93]:
# df1 = df1.rename(columns={0:'Drug'})
# df2 = df2.rename(columns={0:'Drug'})
# df3 = df3.rename(columns={0:'Drug'})
# df4 = df4.rename(columns={0:'Drug'})
# df5 = df5.rename(columns={0:'Drug'})
# df6 = df6.rename(columns={0:'Drug'})
# df7 = df7.rename(columns={0:'Drug'})
# df8 = df8.rename(columns={0:'Drug'})
# df9 = df9.rename(columns={0:'Drug'})
# df10 = df10.rename(columns={0:'Drug'})

In [94]:
# df1['Target'] = gaba
# df2['Target'] = gaba
# df3['Target'] = gaba
# df4['Target'] = gaba
# df5['Target'] = gaba
# df6['Target'] = gaba
# df7['Target'] = gaba
# df8['Target'] = gaba
# df9['Target'] = gaba
# df10['Target'] = gaba

In [95]:
# vdf1['Target'] = vdr
# vdf2['Target'] = vdr
# vdf3['Target'] = vdr
# vdf4['Target'] = vdr
# vdf5['Target'] = vdr
# vdf6['Target'] = vdr
# vdf7['Target'] = vdr
# vdf8['Target'] = vdr
# vdf9['Target'] = vdr
# vdf10['Target'] = vdr

In [14]:
vdf1

,Drug,Target
0,CC(CCCC(C)(C)O)C1CCC2C1(CCCC2=CC=C3CC(CC(C3=C)...,MEAMAASTSLPDPGDFDRNVPRICGVCGDRATGFHFNAMTCEGCKG...
1,C=C1C(=CC=C2CCCC3(C)C2CCC3C(C)C(C)CCCC(C)(C)O)...,MEAMAASTSLPDPGDFDRNVPRICGVCGDRATGFHFNAMTCEGCKG...
2,C=C1C(=CC=C2CCCC3(C)C2CCC3C(CC)CCCC(C)(C)O)CC(...,MEAMAASTSLPDPGDFDRNVPRICGVCGDRATGFHFNAMTCEGCKG...
3,C=C1C(=CC=C2CCCC3(C)C2CCC3C(C)CCCC(C)(C)O)CC(O...,MEAMAASTSLPDPGDFDRNVPRICGVCGDRATGFHFNAMTCEGCKG...
4,C=C1C(=CC=C2CCCC3(C)C2CCC3C(C)CCCCC(C)(C)O)CC(...,MEAMAASTSLPDPGDFDRNVPRICGVCGDRATGFHFNAMTCEGCKG...
...,...,...
788,CCC(CC)(c1ccc(C(CC)(CC)c2ccc(OC(C)CO)c(C)c2)cc...,MEAMAASTSLPDPGDFDRNVPRICGVCGDRATGFHFNAMTCEGCKG...
789,CCC(C)(O)c1ccc(C(CC)(CC)c2ccc(C(O)(CC)CC)cc2)cc1C,MEAMAASTSLPDPGDFDRNVPRICGVCGDRATGFHFNAMTCEGCKG...
790,CCC(CC)(c1ccccc1)c1ccc(OCC(O)C(C)(C)C)cc1,MEAMAASTSLPDPGDFDRNVPRICGVCGDRATGFHFNAMTCEGCKG...
791,CCC(O)(CC)c1ccc(C(C)(CC)c2ccc(OCCCO)cc2)cc1,MEAMAASTSLPDPGDFDRNVPRICGVCGDRATGFHFNAMTCEGCKG...


Add reference smile to the top of the first df

In [141]:
vdf1.loc[-1] = [calcitriol,vdr,0.0]

vdf1.index = vdf1.index + 1

vdf1 = vdf1.sort_index()

In [39]:
df1['Y'] = np.zeros(len(df1))
df2['Y'] = np.zeros(len(df2))
df3['Y'] = np.zeros(len(df3))
df4['Y'] = np.zeros(len(df4))
df5['Y'] = np.zeros(len(df5))
df6['Y'] = np.zeros(len(df6))
df7['Y'] = np.zeros(len(df7))
df8['Y'] = np.zeros(len(df8))
df9['Y'] = np.zeros(len(df9))
df10['Y'] = np.zeros(len(df10))

In [15]:
vdf1['Y'] = np.zeros(len(vdf1))
vdf2['Y'] = np.zeros(len(vdf2))
vdf3['Y'] = np.zeros(len(vdf3))
vdf4['Y'] = np.zeros(len(vdf4))
vdf5['Y'] = np.zeros(len(vdf5))
vdf6['Y'] = np.zeros(len(vdf6))
vdf7['Y'] = np.zeros(len(vdf7))
vdf8['Y'] = np.zeros(len(vdf8))
vdf9['Y'] = np.zeros(len(vdf9))
vdf10['Y'] = np.zeros(len(vdf10))

In [142]:
vdf1

,Drug,Target,Y
0,CC(CCCC(C)(C)O)C1CCC2C1(CCCC2=CC=C3CC(CC(C3=C)...,MEAMAASTSLPDPGDFDRNVPRICGVCGDRATGFHFNAMTCEGCKG...,0.0
1,CN1C(=O)CN=C(c2ccccc2F)c2cc(Cl)ccc21,MRKSPGLSDCLWAWILLLSTLTGRSYGQPSLQDELKDNTTVFTRIL...,0.0
2,O=C1CN=C(c2ccccc2)c2cc(Cl)ccc2N1,MRKSPGLSDCLWAWILLLSTLTGRSYGQPSLQDELKDNTTVFTRIL...,0.0
3,CN1CCN=C(c2ccccc2)c2cc(Cl)ccc21,MRKSPGLSDCLWAWILLLSTLTGRSYGQPSLQDELKDNTTVFTRIL...,0.0
4,CN1C(=O)CN=C(c2ccccc2)c2cc(N)ccc21,MRKSPGLSDCLWAWILLLSTLTGRSYGQPSLQDELKDNTTVFTRIL...,0.0
...,...,...,...
484,NC1=NC(C=Cc2ccc3nccnc3c2)CCC1,MRKSPGLSDCLWAWILLLSTLTGRSYGQPSLQDELKDNTTVFTRIL...,0.0
485,O=C(OC1CC1)c1nccc2cc(-c3ccccc3)ccc12,MRKSPGLSDCLWAWILLLSTLTGRSYGQPSLQDELKDNTTVFTRIL...,0.0
486,O=C1NCC1=COc1ccc2cncnc2c1,MRKSPGLSDCLWAWILLLSTLTGRSYGQPSLQDELKDNTTVFTRIL...,0.0
487,COC1CN=c2ccc(Cl)cc2=N1,MRKSPGLSDCLWAWILLLSTLTGRSYGQPSLQDELKDNTTVFTRIL...,0.0


In [18]:
m1[61:-4]

'LOWDATAREGIME_molecules_10_0.7_VDR_TOP_SIMILAR'

In [20]:
vdf1.to_csv('VitaminDZinc-structures-filteres-by-similarity-to-calcitriol/csv_format/'+m1[61:-4]+'.csv', index = False)
vdf2.to_csv('VitaminDZinc-structures-filteres-by-similarity-to-calcitriol/csv_format/'+m2[61:-4]+'.csv', index = False)
vdf3.to_csv('VitaminDZinc-structures-filteres-by-similarity-to-calcitriol/csv_format/'+m3[61:-4]+'.csv', index = False)
vdf4.to_csv('VitaminDZinc-structures-filteres-by-similarity-to-calcitriol/csv_format/'+m4[61:-4]+'.csv', index = False)
vdf5.to_csv('VitaminDZinc-structures-filteres-by-similarity-to-calcitriol/csv_format/'+m5[61:-4]+'.csv', index = False)
vdf6.to_csv('VitaminDZinc-structures-filteres-by-similarity-to-calcitriol/csv_format/'+m6[61:-4]+'.csv', index = False)
vdf7.to_csv('VitaminDZinc-structures-filteres-by-similarity-to-calcitriol/csv_format/'+m7[61:-4]+'.csv', index = False)
vdf8.to_csv('VitaminDZinc-structures-filteres-by-similarity-to-calcitriol/csv_format/'+m8[61:-4]+'.csv', index = False)
vdf9.to_csv('VitaminDZinc-structures-filteres-by-similarity-to-calcitriol/csv_format/'+m9[61:-4]+'.csv', index = False)
vdf10.to_csv('VitaminDZinc-structures-filteres-by-similarity-to-calcitriol/csv_format/'+m10[61:-4]+'.csv', index = False)


In [96]:
def atom_features(atom):
    return np.array(one_of_k_encoding_unk(atom.GetSymbol(),['C', 'N', 'O', 'S', 'F', 'Si', 'P', 'Cl', 'Br', 'Mg', 'Na','Ca', 'Fe', 'As', 'Al', 'I', 'B', 'V', 'K', 'Tl', 'Yb','Sb', 'Sn', 'Ag', 'Pd', 'Co', 'Se', 'Ti', 'Zn', 'H','Li', 'Ge', 'Cu', 'Au', 'Ni', 'Cd', 'In', 'Mn', 'Zr','Cr', 'Pt', 'Hg', 'Pb', 'Unknown']) +
                    one_of_k_encoding(atom.GetDegree(), [0, 1, 2, 3, 4, 5, 6,7,8,9,10]) +
                    one_of_k_encoding_unk(atom.GetTotalNumHs(), [0, 1, 2, 3, 4, 5, 6,7,8,9,10]) +
                    one_of_k_encoding_unk(atom.GetImplicitValence(), [0, 1, 2, 3, 4, 5, 6,7,8,9,10]) +
                    [atom.GetIsAromatic()])

def one_of_k_encoding(x, allowable_set):
    if x not in allowable_set:
        raise Exception("input {0} not in allowable set{1}:".format(x, allowable_set))
    return list(map(lambda s: x == s, allowable_set))

def one_of_k_encoding_unk(x, allowable_set):
    """Maps inputs not in the allowable set to the last element."""
    if x not in allowable_set:
        x = allowable_set[-1]
    return list(map(lambda s: x == s, allowable_set))

def smile_to_graph(smile):
    mol = Chem.MolFromSmiles(smile)
    
    c_size = mol.GetNumAtoms()
    
    features = []
    for atom in mol.GetAtoms():
        feature = atom_features(atom)
        features.append( feature / sum(feature) )

    edges = []
    for bond in mol.GetBonds():
        edges.append([bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()])
    g = nx.Graph(edges).to_directed()
    edge_index = []
    for e1, e2 in g.edges:
        edge_index.append([e1, e2])
        
    return c_size, features, edge_index

def seq_cat(prot):
    x = np.zeros(max_seq_len)
    for i, ch in enumerate(prot[:max_seq_len]): 
        x[i] = seq_dict[ch]
    return x 

seq_voc = "ABCDEFGHIKLMNOPQRSTUVWXYZ"
seq_dict = {v:(i+1) for i,v in enumerate(seq_voc)}
seq_dict_len = len(seq_dict)
max_seq_len = 1000

In [97]:
dfs = [vdf1, vdf2, vdf3, vdf4, vdf5, vdf6, vdf7, vdf8, vdf9, vdf10]

In [98]:
dfs2 = [df1, df2, df3, df4, df5, df6, df7, df8, df9, df10]

In [99]:
file_names = [m1[61:-4], m2[61:-4], m3[61:-4], m4[61:-4], m5[61:-4], m6[61:-4], m7[61:-4], m8[61:-4], m9[61:-4], m10[61:-4]]

In [100]:
file_names2 = [f1[127:-4], f2[127:-4], f3[127:-4], f4[127:-4], f5[127:-4], f6[127:-4], f7[127:-4], f8[127:-4], f9[127:-4], f10[127:-4]]

Convert to pytorch format

In [24]:
smile_graph_list = list()
for df in dfs:
    compound_iso_smiles = []
    compound_iso_smiles = list( df['Drug'] )
    compound_iso_smiles = set(compound_iso_smiles)
    smile_graph = {}
    for smile in tqdm(compound_iso_smiles):
        g = smile_to_graph(smile)
        smile_graph[smile] = g
    smile_graph_list.append(smile_graph)


100%|██████████| 1000/1000 [00:06<00:00, 146.12it/s]


In [101]:
def predicting(model, device, loader):
    model.eval()
    total_preds = torch.Tensor()
    print('Make prediction for {} samples...'.format(len(loader.dataset)))
    with torch.no_grad():
        for data in tqdm(loader):
            data = data.to(device)
            output = model(data)
            total_preds = torch.cat((total_preds.to(device), output), 0)
    return total_preds.cpu().numpy().flatten()

In [26]:
# convert to PyTorch data format
name = 1
indx= 0
for df in dfs:
    processed_data_file = 'data/processed/vdf' + str(name) + '.pt'
    if ((not os.path.isfile(processed_data_file))):
        train_drugs, train_prots,  train_Y = list(df['Drug']),list(df['Target']),list(df['Y'])
        XT = [seq_cat(t) for t in train_prots]
        train_drugs, train_prots,  train_Y = np.asarray(train_drugs), np.asarray(XT), np.asarray(train_Y)
        # make data PyTorch Geometric ready
        print('preparing df' +str(name)+ '.pt in pytorch format!')
        train_data = TestbedDataset(root='VitaminDZinc-structures-filteres-by-similarity-to-calcitriol/csv_format', dataset=file_names[indx], xd=train_drugs, xt=train_prots, y=train_Y,smile_graph=smile_graph_list[indx])
        print(processed_data_file +' have been created')        
    else:
        print(processed_data_file, ' are already created')
    name += 1
    indx +=1

preparing df1.pt in pytorch format!
Pre-processed data VitaminDZinc-structures-filteres-by-similarity-to-calcitriol/csv_format/processed/LOWDATAREGIME_molecules_10_0.7_VDR_TOP_SIMILAR.pt not found, doing pre-processing...
Converting SMILES to graph: 1/793
Converting SMILES to graph: 2/793
Converting SMILES to graph: 3/793
Converting SMILES to graph: 4/793
Converting SMILES to graph: 5/793
Converting SMILES to graph: 6/793
Converting SMILES to graph: 7/793
Converting SMILES to graph: 8/793
Converting SMILES to graph: 9/793
Converting SMILES to graph: 10/793
Converting SMILES to graph: 11/793
Converting SMILES to graph: 12/793
Converting SMILES to graph: 13/793
Converting SMILES to graph: 14/793
Converting SMILES to graph: 15/793
Converting SMILES to graph: 16/793
Converting SMILES to graph: 17/793
Converting SMILES to graph: 18/793
Converting SMILES to graph: 19/793
Converting SMILES to graph: 20/793
Converting SMILES to graph: 21/793
Converting SMILES to graph: 22/793
Converting SMILES

In [120]:
md1 = '../GraphDTA_Results/Davis/GAT_GCN_Davis/model_GAT_GCN_davis.model'
md2 = '../GraphDTA_Results/Davis/GATNet_Davis/model_GATNet_davis.model'
md3 = '../GraphDTA_Results/Davis/GATNet_Davis/model_GATNet_davis2.model'
md4 = '../GraphDTA_Results/Davis/GCNNet_Davis/model_GCNNet_davis.model'
md5 = '../GraphDTA_Results/Davis/GINConvNet_Davis/model_GINConvNet_davis.model'


md6 = '../GraphDTA_Results/KIBA/GAT_GCN_KIBA/model_GAT_GCN_kiba.model'
md7 = '../GraphDTA_Results/KIBA/GATNet_KIBA/model_GATNet_kiba.model'
md8 = '../GraphDTA_Results/KIBA/GCNNet_KIBA/model_GCNNet_kiba.model'
md9 = '../GraphDTA_Results/KIBA/GINConvNet_KIBA/model_GINConvNet_kiba.model'

md10 = '../GraphDTA_Results/BindingDB/GAT_GCN/model_GAT_GCN_Kd.model'
md11 = '../GraphDTA_Results/BindingDB/GATNet/model_GATNet_bdtdc_ic50.model'
md12 = '../GraphDTA_Results/BindingDB/GATNet/model_GATNet_IC50.model'
md13 = '../GraphDTA_Results/BindingDB/GATNet/model_GATNet_Kd.model'

md14 = '../GraphDTA_Results/BindingDB/GCNNet/bdtdc_ic50_dec20/model_GCNNet_bdtdc_ic50.model'
md15 = '../GraphDTA_Results/BindingDB/GCNNet/strange_model_try_to_test/model_GCNNet_Kd.model'
md16 = '../GraphDTA_Results/BindingDB/GCNNet/model_GCNNet_bdtdc_ic50.model'
md17 = '../GraphDTA_Results/BindingDB/GCNNet/model_GCNNet_bdtdc_kd.model'

md18 = '../GraphDTA_Results/BindingDB/GCNNet/model_GCNNet_bdtdc_ki.model'
md19 = '../GraphDTA_Results/BindingDB/GINConvNet/model_GINConvNet_bindingdb_Kd.model'
md20 = '../GraphDTA_Results/BindingDB/GINConvNet/model_GINConvNet_IC50.model'
md21 = '../GraphDTA_Results/BindingDB/GINConvNet/model_GINConvNet_Ki.model'

md22 = '../GraphDTA_Results/BindingDB/GAT_GCN/model_GAT_GCN_bdtdc_ki.model'
md23 = '../GraphDTA_Results/BindingDB/GAT_GCN/model_GAT_GCN_bdtdc_ic50.model'

md24 = '../GraphDTA_Results/BindingDB/GCNNet/model_GCNNet_bindingdb_kd.model'


In [121]:
models_list = [md1, md2, md3, md4,md5, md6, md7, md8,md9, md10, md11, md12,md13, md14, md15, md16, md17, md18, md19, md20,md21, md22,md23]
model_names = [md1[46:-6], md2[45:-6],'GATNet_davis2', 'GCNNet_davis', 'GINConvNet_davis', 'GAT_GCN_kiba', 'GATNet_kiba', 'GCNNet_kiba', 'GINConvNet_kiba', 'GAT_GCN_Kd', 'GATNet_bdtdc_ic50', 'GATNet_IC50', 'GATNet_Kd', 'bdtdc_ic50_dec20/model_GCNNet_bdtdc_ic50','strange_model_try_to_test/model_GCNNet_Kd', 'GCNNet_bdtdc_ic50', 'GCNNet_bdtdc_kd', 'GCNNet_bdtdc_ki', 'GINConvNet_bindingdb_Kd', 'GINConvNet_IC50', 'GINConvNet_Ki', 'GAT_GCN_bdtdc_ki_latest', 'GAT_GCN_bdtdc_ic50_latest' ]
model_arch = [GAT_GCN, GATNet,GATNet, GCNNet, GINConvNet, GAT_GCN, GATNet, GCNNet, GINConvNet, GAT_GCN, GATNet, GATNet, GATNet, GCNNet,GCNNet, GCNNet, GCNNet, GCNNet, GINConvNet, GINConvNet, GINConvNet,GAT_GCN,GAT_GCN  ]

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else "cpu")
modeling = model_arch[-1]
model = modeling()
model.to(device)
model.load_state_dict(torch.load(models_list[-1],map_location='cuda:0'))

TEST_BATCH_SIZE = 512
indx= 0
for df in dfs:
    test_data = TestbedDataset(root='Benzos-structures-filteres-by-similarity-to-diazepam-20211216T105015Z-001/Benzos-structures-filteres-by-similarity-to-diazepam/csv_format', dataset=file_names[indx])
    test_loader = DataLoader(test_data, batch_size=TEST_BATCH_SIZE, shuffle=False)
    pred = predicting(model, device, test_loader)
    df[model_names[i]] = pred
    indx+=1

In [110]:
dfs[0]

,Drug,Target,Y
0,CN1C(=O)CN=C(c2ccccc2F)c2cc(Cl)ccc21,MRKSPGLSDCLWAWILLLSTLTGRSYGQPSLQDELKDNTTVFTRIL...,0.0
1,O=C1CN=C(c2ccccc2)c2cc(Cl)ccc2N1,MRKSPGLSDCLWAWILLLSTLTGRSYGQPSLQDELKDNTTVFTRIL...,0.0
2,CN1CCN=C(c2ccccc2)c2cc(Cl)ccc21,MRKSPGLSDCLWAWILLLSTLTGRSYGQPSLQDELKDNTTVFTRIL...,0.0
3,CN1C(=O)CN=C(c2ccccc2)c2cc(N)ccc21,MRKSPGLSDCLWAWILLLSTLTGRSYGQPSLQDELKDNTTVFTRIL...,0.0
4,CN1C(=O)CCN=C(c2ccccc2)c2cc(Cl)ccc21,MRKSPGLSDCLWAWILLLSTLTGRSYGQPSLQDELKDNTTVFTRIL...,0.0
...,...,...,...
483,NC1=NC(C=Cc2ccc3nccnc3c2)CCC1,MRKSPGLSDCLWAWILLLSTLTGRSYGQPSLQDELKDNTTVFTRIL...,0.0
484,O=C(OC1CC1)c1nccc2cc(-c3ccccc3)ccc12,MRKSPGLSDCLWAWILLLSTLTGRSYGQPSLQDELKDNTTVFTRIL...,0.0
485,O=C1NCC1=COc1ccc2cncnc2c1,MRKSPGLSDCLWAWILLLSTLTGRSYGQPSLQDELKDNTTVFTRIL...,0.0
486,COC1CN=c2ccc(Cl)cc2=N1,MRKSPGLSDCLWAWILLLSTLTGRSYGQPSLQDELKDNTTVFTRIL...,0.0


In [128]:
file_names

['LOWDATAREGIME_molecules_10_0.7_VDR_TOP_SIMILAR',
 'LOWDATAREGIME_molecules_20_0.7_VDR_TOP_SIMILAR',
 'LOWDATAREGIME_molecules_30_0.7_VDR_TOP_SIMILAR',
 'LOWDATAREGIME_molecules_40_0.7_VDR_TOP_SIMILAR',
 'REINVENT_VDR_TOP_SIMILAR',
 'TRANSMOL_VDR_TOP_SIMILAR',
 'TRANSVAE_vdr_pubchem_k_dims_VDR_TOP_SIMILAR',
 'TRANSVAE_vdr_pubchem_top_dims_VDR_TOP_SIMILAR',
 'TRANSVAE_vdr_zinc_k_dims_VDR_TOP_SIMILAR',
 'TRANSVAE_vdr_zinc_top_dims_VDR_TOP_SIMILAR']

In [123]:
for i in range(len(models_list)):
    device = torch.device('cuda:0' if torch.cuda.is_available() else "cpu")
    modeling = model_arch[i]
    model = modeling()
    model.to(device)
    model.load_state_dict(torch.load(models_list[i],map_location='cuda:0'))

    TEST_BATCH_SIZE = 512
    indx= 0
    for df in dfs2:
        test_data = TestbedDataset(root='VitaminDZinc-structures-filteres-by-similarity-to-calcitriol/csv_format', dataset=file_names[indx])
        test_loader = DataLoader(test_data, batch_size=TEST_BATCH_SIZE, shuffle=False)
        pred = predicting(model, device, test_loader)
        df[model_names[i]] = pred
        indx+=1

    

Pre-processed data found: VitaminDZinc-structures-filteres-by-similarity-to-calcitriol/csv_format/processed/LOWDATAREGIME_molecules_10_0.7_VDR_TOP_SIMILAR.pt, loading ...
Make prediction for 793 samples...


100%|██████████| 2/2 [00:00<00:00,  4.85it/s]


Pre-processed data found: VitaminDZinc-structures-filteres-by-similarity-to-calcitriol/csv_format/processed/LOWDATAREGIME_molecules_20_0.7_VDR_TOP_SIMILAR.pt, loading ...
Make prediction for 744 samples...


100%|██████████| 2/2 [00:00<00:00,  6.88it/s]


Pre-processed data found: VitaminDZinc-structures-filteres-by-similarity-to-calcitriol/csv_format/processed/LOWDATAREGIME_molecules_30_0.7_VDR_TOP_SIMILAR.pt, loading ...
Make prediction for 665 samples...


...

100%|██████████| 2/2 [00:00<00:00,  4.54it/s]


Pre-processed data found: VitaminDZinc-structures-filteres-by-similarity-to-calcitriol/csv_format/processed/TRANSVAE_vdr_zinc_k_dims_VDR_TOP_SIMILAR.pt, loading ...
Make prediction for 1000 samples...


100%|██████████| 2/2 [00:00<00:00,  4.38it/s]


Pre-processed data found: VitaminDZinc-structures-filteres-by-similarity-to-calcitriol/csv_format/processed/TRANSVAE_vdr_zinc_top_dims_VDR_TOP_SIMILAR.pt, loading ...
Make prediction for 1000 samples...


100%|██████████| 2/2 [00:00<00:00,  4.29it/s]


In [52]:
for df in dfs:
    df = df.drop(columns=['Y'])

In [118]:
dfs[0] = dfs[0].drop(columns=['Y'])
dfs[1] = dfs[1].drop(columns=['Y'])
dfs[2] = dfs[2].drop(columns=['Y'])
dfs[3] = dfs[3].drop(columns=['Y'])
dfs[4] = dfs[4].drop(columns=['Y'])
dfs[5] = dfs[5].drop(columns=['Y'])
dfs[6] = dfs[6].drop(columns=['Y'])
dfs[7] = dfs[7].drop(columns=['Y'])
dfs[8] = dfs[8].drop(columns=['Y'])
dfs[9] = dfs[9].drop(columns=['Y'])

In [114]:
dfs2[0] = dfs2[0].drop(columns=['Y'])
dfs2[1] = dfs2[1].drop(columns=['Y'])
dfs2[2] = dfs2[2].drop(columns=['Y'])
dfs2[3] = dfs2[3].drop(columns=['Y'])
dfs2[4] = dfs2[4].drop(columns=['Y'])
dfs2[5] = dfs2[5].drop(columns=['Y'])
dfs2[6] = dfs2[6].drop(columns=['Y'])
dfs2[7] = dfs2[7].drop(columns=['Y'])
dfs2[8] = dfs2[8].drop(columns=['Y'])
dfs2[9] = dfs2[9].drop(columns=['Y'])

In [125]:
dfs[0]

,Drug,Target,GAT_GCN_davis,GATNet_davis,GATNet_davis2,GCNNet_davis,GINConvNet_davis,GAT_GCN_kiba,GATNet_kiba,GCNNet_kiba,...,bdtdc_ic50_dec20/model_GCNNet_bdtdc_ic50,strange_model_try_to_test/model_GCNNet_Kd,GCNNet_bdtdc_ic50,GCNNet_bdtdc_kd,GCNNet_bdtdc_ki,GINConvNet_bindingdb_Kd,GINConvNet_IC50,GINConvNet_Ki,GAT_GCN_bdtdc_ki_latest,GAT_GCN_bdtdc_ic50_latest
0,CN1C(=O)CN=C(c2ccccc2F)c2cc(Cl)ccc21,MRKSPGLSDCLWAWILLLSTLTGRSYGQPSLQDELKDNTTVFTRIL...,5.764262,5.693633,5.844633,5.790274,5.317925,11.537140,11.585246,10.905057,...,4.587414,7.954102,6.004145,6.584356,8.035607,9.000124,6.530537,8.019577,6.781021,4.908838
1,O=C1CN=C(c2ccccc2)c2cc(Cl)ccc2N1,MRKSPGLSDCLWAWILLLSTLTGRSYGQPSLQDELKDNTTVFTRIL...,5.820510,5.347934,5.088555,5.706682,5.599360,11.650953,11.413750,10.463663,...,4.379277,8.154282,5.655460,6.472925,8.157581,8.192932,4.286624,6.693745,7.465182,5.336170
2,CN1CCN=C(c2ccccc2)c2cc(Cl)ccc21,MRKSPGLSDCLWAWILLLSTLTGRSYGQPSLQDELKDNTTVFTRIL...,5.781652,6.222953,5.784204,5.979016,5.766552,11.689509,11.376568,10.623102,...,4.212502,7.276699,5.236395,6.691364,7.730948,8.729960,5.920058,7.680098,7.867999,5.522687
3,CN1C(=O)CN=C(c2ccccc2)c2cc(N)ccc21,MRKSPGLSDCLWAWILLLSTLTGRSYGQPSLQDELKDNTTVFTRIL...,5.703455,5.775535,5.232235,5.181041,5.449607,11.536312,11.222670,11.156598,...,4.195644,7.209634,5.302639,7.317929,7.221502,7.887417,5.791874,7.391438,6.965907,4.801177
4,CN1C(=O)CCN=C(c2ccccc2)c2cc(Cl)ccc21,MRKSPGLSDCLWAWILLLSTLTGRSYGQPSLQDELKDNTTVFTRIL...,5.347439,6.370338,5.859731,5.626638,5.456227,11.525576,11.430586,10.467786,...,4.553962,7.454924,5.320091,7.266023,7.910477,8.702001,5.418643,7.303138,7.689903,5.136808
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
483,NC1=NC(C=Cc2ccc3nccnc3c2)CCC1,MRKSPGLSDCLWAWILLLSTLTGRSYGQPSLQDELKDNTTVFTRIL...,5.988225,6.207549,5.812002,5.292656,5.809867,11.671762,11.476151,10.881434,...,6.054760,8.888521,7.458836,6.216098,7.743773,9.002476,5.277848,8.199919,6.887542,6.513785
484,O=C(OC1CC1)c1nccc2cc(-c3ccccc3)ccc12,MRKSPGLSDCLWAWILLLSTLTGRSYGQPSLQDELKDNTTVFTRIL...,6.026775,5.510842,5.401698,5.795946,5.684030,11.552782,11.378396,10.424806,...,5.602536,9.047976,6.038068,6.343800,7.501207,7.969134,6.865233,7.098046,6.925295,5.284789
485,O=C1NCC1=COc1ccc2cncnc2c1,MRKSPGLSDCLWAWILLLSTLTGRSYGQPSLQDELKDNTTVFTRIL...,5.160220,5.217212,5.890978,5.327335,5.549436,11.563120,11.498323,10.587861,...,6.576860,8.319893,6.734782,5.972180,6.145973,8.368226,6.189260,8.255099,6.086030,6.323635
486,COC1CN=c2ccc(Cl)cc2=N1,MRKSPGLSDCLWAWILLLSTLTGRSYGQPSLQDELKDNTTVFTRIL...,5.912923,6.358792,5.317985,5.248207,5.167810,11.467406,11.234021,10.676084,...,4.306296,8.607360,5.245682,5.692453,6.761212,9.309616,4.699046,6.586902,7.038149,5.808917


In [131]:
idx = 0
for df in dfs:
    df.to_csv('prediction_for_generated_benzos_all/GraphDTA_prediction_'+file_names2[idx]+'.csv',index= False )
    idx +=1

In [132]:
file_names2

['LOWDATAREGIME_molecules_10_0.7_BENZOS_TOP_SIMILAR',
 'LOWDATAREGIME_molecules_20_0.7_BENZOS_TOP_SIMILAR',
 'LOWDATAREGIME_molecules_30_0.7_BENZOS_TOP_SIMILAR',
 'LOWDATAREGIME_molecules_40_0.7_BENZOS_TOP_SIMILAR',
 'REINVENT_BENZOS_TOP_SIMILAR',
 'TRANSMOL_BENZOS_TOP_SIMILAR',
 'TRANSVAE_benzos_pubchem_k_dims_BENZOS_TOP_SIMILAR',
 'TRANSVAE_benzos_pubchem_top_dims_BENZOS_TOP_SIMILAR',
 'TRANSVAE_benzos_zinc_k_dims_BENZOS_TOP_SIMILAR',
 'TRANSVAE_benzos_zinc_top_dims_BENZOS_TOP_SIMILAR']

In [98]:
name = 1
indx= 0
results = {}
for df in dfs:
    test_data = TestbedDataset(root='VitaminDZinc-structures-filteres-by-similarity-to-calcitriol/csv_format', dataset=file_names[indx])
    test_loader = DataLoader(test_data, batch_size=TEST_BATCH_SIZE, shuffle=False)
    pred = predicting(model, device, test_loader)
    results['vdf'+str(name)] = pred
    name += 1
    indx += 1

  0%|          | 0/1 [00:00<?, ?it/s]

Pre-processed data found: Benzos-structures-filteres-by-similarity-to-diazepam-20211216T105015Z-001/Benzos-structures-filteres-by-similarity-to-diazepam/csv_format/processed/LOWDATAREGIME_molecules_10_0.7_BENZOS_TOP_SIMILAR.pt, loading ...
Make prediction for 488 samples...
Pre-processed data found: Benzos-structures-filteres-by-similarity-to-diazepam-20211216T105015Z-001/Benzos-structures-filteres-by-similarity-to-diazepam/csv_format/processed/LOWDATAREGIME_molecules_20_0.7_BENZOS_TOP_SIMILAR.pt, loading ...
Make prediction for 475 samples...


  0%|          | 0/1 [00:00<?, ?it/s]

Pre-processed data found: Benzos-structures-filteres-by-similarity-to-diazepam-20211216T105015Z-001/Benzos-structures-filteres-by-similarity-to-diazepam/csv_format/processed/LOWDATAREGIME_molecules_30_0.7_BENZOS_TOP_SIMILAR.pt, loading ...
Make prediction for 394 samples...
Pre-processed data found: Benzos-structures-filteres-by-similarity-to-diazepam-20211216T105015Z-001/Benzos-structures-filteres-by-similarity-to-diazepam/csv_format/processed/LOWDATAREGIME_molecules_40_0.7_BENZOS_TOP_SIMILAR.pt, loading ...
Make prediction for 351 samples...


  0%|          | 0/2 [00:00<?, ?it/s]

...

100%|██████████| 2/2 [00:00<00:00, 10.55it/s]

Pre-processed data found: Benzos-structures-filteres-by-similarity-to-diazepam-20211216T105015Z-001/Benzos-structures-filteres-by-similarity-to-diazepam/csv_format/processed/TRANSVAE_benzos_pubchem_top_dims_BENZOS_TOP_SIMILAR.pt, loading ...
Make prediction for 1000 samples...



  0%|          | 0/2 [00:00<?, ?it/s]

Pre-processed data found: Benzos-structures-filteres-by-similarity-to-diazepam-20211216T105015Z-001/Benzos-structures-filteres-by-similarity-to-diazepam/csv_format/processed/TRANSVAE_benzos_zinc_k_dims_BENZOS_TOP_SIMILAR.pt, loading ...
Make prediction for 1000 samples...
Pre-processed data found: Benzos-structures-filteres-by-similarity-to-diazepam-20211216T105015Z-001/Benzos-structures-filteres-by-similarity-to-diazepam/csv_format/processed/TRANSVAE_benzos_zinc_top_dims_BENZOS_TOP_SIMILAR.pt, loading ...
Make prediction for 1000 samples...


100%|██████████| 2/2 [00:00<00:00, 11.74it/s]


In [99]:
results['df7']

array([7.471857 , 6.819584 , 6.819585 , 6.8195844, 6.819584 , 7.3719416,
       7.3599763, 6.5618935, 6.5618935, 6.5618935, 7.807021 , 6.8007584,
       6.2966256, 6.2966256, 7.476516 , 6.710307 , 6.347851 , 6.208635 ,
       5.7133207, 7.078358 , 7.0317507, 7.96808  , 6.351198 , 7.4227753,
       7.4227757, 7.4227753, 7.4227757, 7.4227753, 7.422776 , 7.4227753,
       7.4227757, 7.4227753, 7.4227753, 7.4227753, 7.4227757, 7.4227753,
       7.4227757, 7.4227753, 7.4227757, 7.4227753, 7.4227757, 7.4227753,
       7.4227753, 7.422775 , 7.422776 , 7.4227753, 7.4227757, 7.4227753,
       7.4227757, 7.4227753, 7.4227757, 7.4227753, 7.4227757, 7.4227753,
       7.4227757, 7.4227753, 7.3074007, 7.307401 , 7.3074017, 7.307401 ,
       7.102207 , 7.102207 , 7.1022077, 8.257356 , 8.257356 , 8.257356 ,
       8.257356 , 8.257356 , 8.257356 , 8.257356 , 8.257356 , 5.996225 ,
       7.5002494, 6.6254973, 7.4486165, 7.649945 , 8.299431 , 7.758526 ,
       8.315676 , 5.765329 , 7.2700505, 8.297096 , 

In [100]:
idx = 0
for df in dfs:
    df['Ki_log1p_bindingDB'] = results['df'+str(idx+1)]
    idx +=1

In [102]:
for df in dfs:
    df = df.drop(columns=['Y'])


In [107]:
idx = 0
for df in dfs:
    df.to_csv('prediction_for_generated/GINConvNet_prediction_'+file_names[idx]+'.csv',index= False )
    idx +=1

In [ ]:
# Prediction for diazpezam
# kiba_score	Kd_log1p	Kd_log1p_bindingDB	IC50_log1p_bindingDB	Ki_log1p_bindingDB
# 11.478367	    5.4345627	7.686416	        5.6970625	            7.6559587
